In [0]:
import sys, os
sys.path.append(os.path.abspath('/Workspace/Repos/zhastay_yeltay@epam.com/utils/'))

from delta.tables import *
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, IntegerType, DateType, LongType

from init import source_path, working_path, bronze, silver, gold
from init import init_spark
from udfs import * 
init_spark()

from util_logger import init_logger
dbutils.widgets.text('task', "test_logger")
logger = init_logger(dbutils.widgets.get('task'))

## 1. Define TOP-5 cities with the largest number of VIP customers and specify the number of such customers for each of the TOP-5 cities. 

In [0]:
logger.info("SQL operation started to populate/update the table 01_cities_by_vip_customer_count.")

In [0]:
%sql

CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.01_cities_by_vip_customer_count
USING DELTA
AS 
WITH cities_by_vip AS (
  SELECT
    a.city,
    COUNT(DISTINCT c.id) AS total_vip_customers
  FROM
    hive_metastore.zhastay_yeltay_02_silver.addresses AS a
    INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o ON a.id = o.address_id
    INNER JOIN hive_metastore.zhastay_yeltay_02_silver.customers AS c ON o.customer_id = c.id
  WHERE
    c.status = 'VIP'
  GROUP BY
    a.city
)
SELECT
  city,
  total_vip_customers
FROM
  cities_by_vip
ORDER BY
  total_vip_customers DESC;

num_affected_rows,num_inserted_rows


In [0]:
logger.info(f"SQL operation completed successfully. Table 01_cities_by_vip_customer_count has been populated/updated and overwritten successfully.")

## 2. Determine whether there were orders and their total number at the address: any combination of street and house number. 

In [0]:
logger.info("SQL operation started to populate/update the table 02_total_orders_by_address.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.02_total_orders_by_address
USING DELTA
AS 
WITH total_orders_by_address AS (
  SELECT
    a.addressline,
    COUNT(DISTINCT o.id) AS total_orders
  FROM hive_metastore.zhastay_yeltay_02_silver.addresses AS a
  INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
    ON a.id = o.address_id
  GROUP BY
    a.addressline
  ORDER BY
    total_orders DESC
)
SELECT
  *
FROM total_orders_by_address;

In [0]:
logger.info(f"SQL operation completed successfully. Table 02_total_orders_by_address has been populated/updated and overwritten successfully.")

## 3. Get the total number of customers in the system as well as their number breakdown by customers’ kinds and types.

In [0]:
logger.info("SQL operation started to populate/update the table 03_total_customers_by_type_status.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.03_total_customers_by_type_status
USING DELTA
AS 
WITH total_customer AS (
  SELECT
    COALESCE(c.type, 'Total') AS type,
    COUNT(id) FILTER(WHERE c.status = 'Regular') AS Regulars,
    COUNT(id) FILTER(WHERE c.status = 'VIP') AS Vips,
    COUNT(id) AS Total
  FROM hive_metastore.zhastay_yeltay_02_silver.customers AS c
  GROUP BY
    c.type
  WITH ROLLUP
)
SELECT
  *
FROM total_customer
ORDER BY type;

In [0]:
logger.info(f"SQL operation completed successfully. Table 03_total_customers_by_type_status has been populated/updated and overwritten successfully.")

## 4. Get a list of all customers in the system breakdown by their kinds and types.

In [0]:
logger.info("SQL operation started to populate/update the table 04_customer_kinds_types_list.")

In [0]:
%sql

CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.04_customer_kinds_types_list
USING DELTA
AS 
SELECT
  type,
  status,
  array_agg(id) AS ids
FROM hive_metastore.zhastay_yeltay_02_silver.customers
GROUP BY
  type,
  status

In [0]:
logger.info(f"SQL operation completed successfully. Table 04_customer_kinds_types_list has been populated/updated and overwritten successfully.")

## 5. Get a list of affiliate customers who made less than 5 orders in a week. 

In [0]:
logger.info("SQL operation started to populate/update the table 05_affiliate_by_weekly_orders.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.05_affiliate_by_weekly_orders
USING DELTA
AS 
WITH affiliate_by_weekly_orders AS (
  SELECT
    c.id AS customer_id,
    c.status AS customer_status,
    (CAST(o.created_on AS DATE) - (dayofweek(o.created_on) + 5) % 7) AS monday_of_week,
    COUNT(*) AS weekly_total_orders
  FROM hive_metastore.zhastay_yeltay_02_silver.addresses AS a
  INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
    ON a.id = o.address_id
  INNER JOIN hive_metastore.zhastay_yeltay_02_silver.customers AS c
    ON o.customer_id = c.id
  WHERE
    c.type = 'Affiliate'
  GROUP BY
    c.id,
    c.status,
    monday_of_week
)
SELECT
  customer_id,
  customer_status
FROM affiliate_by_weekly_orders
GROUP BY
  customer_id,
  customer_status
HAVING
  COUNT(*) FILTER(WHERE weekly_total_orders >= 5) = 0

In [0]:
logger.info(f"SQL operation completed successfully. Table 05_affiliate_by_weekly_orders has been populated/updated and overwritten successfully.")

## 6. Get a list of VIP customers and the amount that was paid to each of them as a coupon discount. 

In [0]:
logger.info("SQL operation started to populate/update the table 06_total_vip_customers_coupon.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.06_total_vip_customers_coupon
USING DELTA
AS 
SELECT
  c.id AS customer_id,
  5 * COUNT(DISTINCT o.id) AS total_coupon_amount
FROM hive_metastore.zhastay_yeltay_02_silver.addresses AS a
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
  ON a.id = o.address_id
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.customers AS c
  ON o.customer_id = c.id
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.metropolitan_cities AS mc
  ON a.city = mc.city
    AND a.state = mc.state
WHERE
  c.status = 'VIP'
GROUP BY
  c.id
ORDER BY
  total_coupon_amount DESC

In [0]:
logger.info(f"SQL operation completed successfully. Table 06_total_vip_customers_coupon has been populated/updated and overwritten successfully.")

## 7. Find out the average delivery time to a specific city and the average number of orders to specified cities.

In [0]:
logger.info("SQL operation started to populate/update the table 07_average_time_and_order_count_by_city.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.07_average_time_and_order_count_by_city
USING DELTA
AS
WITH address_orders AS (
    SELECT
        a.state,
        a.city,
        DATE(o.created_on) AS created_date,
        SUM(DATE_DIFF(o.delivered_on, o.created_on)) AS sum_date_diff,
        COUNT(o.id) AS total_orders
    FROM hive_metastore.zhastay_yeltay_02_silver.addresses AS a
    INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
        ON a.id = o.address_id
    GROUP BY
        a.state,
        a.city,
        created_date
)
SELECT
  state,
  city,
  ROUND(SUM(sum_date_diff) / SUM(total_orders), 2) AS avg_delivery_day,
  ROUND(AVG(total_orders), 2) AS avg_orders_per_day
FROM address_orders
GROUP BY
  state,
  city

In [0]:
logger.info(f"SQL operation completed successfully. Table 07_average_time_and_order_count_by_city has been populated/updated and overwritten successfully.")

## 8. Get the overall number and share of orders, delivery time of which exceeds 7 days as well as broken down by the order type. 

In [0]:
logger.info("SQL operation started to populate/update the table 08_order_delays_by_order_type.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.08_order_delays_by_order_type
USING DELTA
AS
SELECT
  CASE WHEN mc.metropolitan IS NOT NULL THEN 'Metropolitan' ELSE 'Regional' END AS order_type,
  COUNT(*) FILTER(WHERE DATE_DIFF(o.delivered_on, o.created_on) > 7) AS total_delayed_orders,
  COUNT(*) AS total_orders,
  ROUND(100 * (COUNT(*) FILTER(WHERE DATE_DIFF(o.delivered_on, o.created_on) > 7))::NUMERIC / COUNT(*), 2) AS delayed_percentage
FROM hive_metastore.zhastay_yeltay_02_silver.orders AS o
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.addresses AS a
    ON o.address_id = a.id
LEFT JOIN hive_metastore.zhastay_yeltay_02_silver.metropolitan_cities as mc
    ON mc.state = a.state
    AND mc.city = a.city
GROUP BY
    order_type
ORDER BY
    delayed_percentage DESC

In [0]:
logger.info(f"SQL operation completed successfully. Table 08_order_delays_by_order_type has been populated/updated and overwritten successfully.")

## 9. For each state-capital city pair get the total number of orders and the number of orders for the whole state.

In [0]:
logger.info("SQL operation started to populate/update the table 09_total_orders_by_state_capital.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.09_total_orders_by_state_capital
USING DELTA
AS
WITH city_total_orders AS (
  SELECT
    a.state,
    a.city,
    COUNT(o.id) AS total_orders,
    SUM(COUNT(o.id)) OVER(PARTITION BY a.state) AS total_orders_by_state
  FROM
    hive_metastore.zhastay_yeltay_02_silver.addresses AS a
    INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o ON a.id = o.address_id
  GROUP BY
    a.state,
    a.city
)
SELECT
  state,
  city,
  total_orders,
  total_orders_by_state
FROM
  city_total_orders
WHERE
  (state, city) IN (
    SELECT
      state,
      city
    FROM
      hive_metastore.zhastay_yeltay_02_silver.state_capital_cities
  )

In [0]:
logger.info(f"SQL operation completed successfully. Table 09_total_orders_by_state_capital has been populated/updated and overwritten successfully.")

## 10. Get complete information on the order and the customer by the given order number. 

In [0]:
logger.info("SQL operation started to populate/update the table 10_complete_order_customer_by_id.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.10_complete_order_customer_by_id
USING DELTA
AS
SELECT
  o.id AS order_id,
  o.created_on AS order_created_on,
  i.descriptions AS item_description,
  i.price AS item_price,
  od.quantity AS item_quantity,
  i.price * od.quantity AS total_item_price,
  SUM(i.price * od.quantity) OVER() AS total_order_price,
  o.customer_id,
  c.type AS customer_type,
  c.status AS customer_status,
  a.addressline AS delivery_addressline,
  a.city AS delivery_city,
  a.state AS delivery_state,
  a.country AS delivery_country,
  mc.metropolitan IS NOT NULL AS is_city_metropolitan,
  scc.state IS NOT NULL AS is_city_state_capital
FROM hive_metastore.zhastay_yeltay_02_silver.orders AS o
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.order_details AS od
  ON od.order_id = o.id
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.items AS i
  ON i.id = od.item_id

INNER JOIN hive_metastore.zhastay_yeltay_02_silver.customers AS c
  ON o.customer_id = c.id

INNER JOIN hive_metastore.zhastay_yeltay_02_silver.addresses AS a
  ON a.id = o.address_id
LEFT JOIN hive_metastore.zhastay_yeltay_02_silver.metropolitan_cities AS mc
  ON a.state = mc.state
  AND a.city = mc.city
LEFT JOIN hive_metastore.zhastay_yeltay_02_silver.state_capital_cities AS scc
  ON a.state = scc.state
  AND a.city = scc.city
WHERE
  o.id = '2987'

In [0]:
logger.info(f"SQL operation completed successfully. Table 10_complete_order_customer_by_id has been populated/updated and overwritten successfully.")

## 11. Get a list of orders for each of the states for one month. 

In [0]:
logger.info("SQL operation started to populate/update the table 11_monthly_orders_by_state.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.11_monthly_orders_by_state
USING DELTA
AS
SELECT
    a.state,
    EXTRACT(YEAR FROM o.created_on) AS order_year,
    EXTRACT(MONTH FROM o.created_on) AS order_month,
    -- COUNT(o.id) AS total_orders,
    ARRAY_AGG(o.id) AS order_ids
FROM hive_metastore.zhastay_yeltay_02_silver.addresses AS a
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
    ON a.id = o.address_id
GROUP BY
    a.state,
    order_year,
    order_month
ORDER BY
    order_year,
    order_month,
    a.state

In [0]:
logger.info(f"SQL operation completed successfully. Table 11_monthly_orders_by_state has been populated/updated and overwritten successfully.")

## 12. Get the number and list of orders made by each affiliate customer. 

In [0]:
logger.info("SQL operation started to populate/update the table 12_affiliate_customer_orders.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.12_affiliate_customer_orders
USING DELTA
AS
SELECT
    c.id AS customer_id,
    COUNT(o.id) AS total_order
FROM hive_metastore.zhastay_yeltay_02_silver.customers AS c
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.orders AS o
    ON o.customer_id = c.id
WHERE
    c.type = 'Affiliate'
GROUP BY
    c.id
ORDER BY
    c.id

In [0]:
logger.info(f"SQL operation completed successfully. Table 12_affiliate_customer_orders has been populated/updated and overwritten successfully.")

## 13. Define the number of orders that were delivered to a specific customer for the entire time and for the certain period. 

In [0]:
logger.info("SQL operation started to populate/update the table 13_customer_total_order_delivered.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.13_customer_total_order_delivered
USING DELTA
AS
SELECT
    c.id,
    o.delivered_on,
    COUNT(*) AS daily_total_orders,
    COUNT(*) OVER(PARTITION BY c.id) AS total_orders
    -- COUNT(*) FILTER(WHERE o.delivered_on BETWEEN '2020-01-01' AND '2020-02-01') AS total_orders_by_certain_period
FROM hive_metastore.zhastay_yeltay_02_silver.orders AS o
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.customers AS c
    ON o.customer_id = c.id
GROUP BY
    c.id,
    o.delivered_on
ORDER BY
    c.id,
    o.delivered_on

In [0]:
logger.info(f"SQL operation completed successfully. Table 13_customer_total_order_delivered has been populated/updated and overwritten successfully.")

## 14. Get a list and the total number of orders shipped on a specific day in a specific city.

In [0]:
logger.info("SQL operation started to populate/update the table 14_daily_orders_by_city.")

In [0]:
%sql
CREATE OR REPLACE TABLE hive_metastore.zhastay_yeltay_03_gold.14_daily_orders_by_city
USING DELTA
AS
SELECT
    a.city,
    o.delivered_on,
    COUNT(*) AS daily_total_orders
FROM hive_metastore.zhastay_yeltay_02_silver.orders AS o
INNER JOIN hive_metastore.zhastay_yeltay_02_silver.addresses AS a
    ON o.address_id = a.id
GROUP BY
    a.city,
    o.delivered_on
ORDER BY
    a.city,
    o.delivered_on

In [0]:
logger.info(f"SQL operation completed successfully. Table 14_daily_orders_by_city has been populated/updated and overwritten successfully.")